In [12]:
import json

def add_length_attribute(input_filepath, output_filepath):
    # Load original JSON data
    with open(input_filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    texts = data.get("all_generated_texts", [])

    # Build new structure with text and length attributes
    new_texts = [{"text": t, "length": len(t)} for t in texts]
    
    # include "id" attribute
    for i, text in enumerate(new_texts):
        text["id"] = i
    
    

    # Create final dict
    output_data = {"texts": new_texts}

    # Write output JSON file
    with open(output_filepath, 'w', encoding='utf-8') as f_out:
        json.dump(output_data, f_out, ensure_ascii=False, indent=2)
        
    # save as a csv file
    with open(output_filepath.replace('.json', '.csv'), 'w', encoding='utf-8') as f_out:
        f_out.write("id,text,length\n")
        for text in new_texts:
            f_out.write(f"{text['id']},{text['text']},{text['length']}\n")
# Print the number of entries processed

    print(f"Processed {len(texts)} entries. Output saved to {output_filepath}")

if __name__ == "__main__":
    input_file = "eval_before.json"   # Change if your file is named differently or in another folder
    output_file = "eval_after_new.json"       # Output filename

    add_length_attribute(input_file, output_file)


Processed 3729 entries. Output saved to eval_after_new.json


In [ ]:
## Read the test.csv file
import pandas as pd

test_df = pd.read_csv("test.csv")


# filter the dataframe to include only language = "igbo", "yoruba", "hausa"
filtered_df = test_df[test_df["language"].isin(["igbo", "yoruba", "hausa"])]

# add "id" attribute which is the index of the dataframe and reset to start from 0
filtered_df.reset_index(drop=True, inplace=True)
filtered_df["id"] = filtered_df.index


#save the filtered dataframe to a csv file
filtered_df.to_csv("test_filtered.csv", index=False, encoding='utf-8')



/tmp/ipykernel_653783/2481388163.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["id"] = filtered_df.index


In [30]:
# Imoort the json files
import json
import pandas as pd
import os

filtered_df = pd.read_json("test_filtered.json", lines=True)
eval_df = pd.read_json("eval_after_new.json")

print("Filtered DataFrame columns:", filtered_df.columns)
print("Eval DataFrame columns:", eval_df.columns)

# merge the two dataframes on the "id" column
merged_df = pd.merge(filtered_df[["language", "audio_path", "id", "transcription"]], eval_df, on="id", how="inner")

# columns renaming
merged_df.rename(columns={"text": "ph-4_generated_text", "transcription": "ground_truth"}, inplace=True)


#save the merged dataframe to a json file
merged_df.to_json("merged_eval_before.json", orient="records", lines=True, force_ascii=False)





Filtered DataFrame columns: Index(['speaker_id', 'transcription', 'audio_path', 'duration', 'language',
       'gender', 'translation', 'domain', 'id'],
      dtype='object')
Eval DataFrame columns: Index(['text', 'length', 'id'], dtype='object')
